# MNIST_2_WL

Modification of the advanced deep learning network from TensorFlow for the WL project.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range
import matplotlib.pyplot as plt
from astropy.io import fits
plt.rcParams['image.cmap'] = 'viridis'
plt.rcParams['image.interpolation'] = 'none'
%matplotlib inline
from IPython import display

In [2]:
# Set the paths to the raw data files
whereami = '/home/jhargis'
path     = 'Dropbox/astroNN/wl_maps/'
#whereami = '/Users/goldston'
#whereami = '/Users/jegpeek'
#path = 'Documents/Weak_Lensing/kmaps_smoothed/'

In [3]:
# Read in the data
train_dataset = pickle.load( open( whereami+"/"+path+"train_dataset.pkl", "rb" ) )
test_dataset  = pickle.load( open( whereami+"/"+path+"test_dataset.pkl", "rb" ) )
valid_dataset = pickle.load( open( whereami+"/"+path+"valid_dataset.pkl", "rb" ) )
train_labels = pickle.load( open( whereami+"/"+path+"train_labels.pkl", "rb" ) )
test_labels = pickle.load( open( whereami+"/"+path+"test_labels.pkl", "rb" ) )
valid_labels = pickle.load( open( whereami+"/"+path+"valid_labels.pkl", "rb" ) )

In [4]:
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

('Training set', (276080, 32, 32, 1), (276080, 2))
('Validation set', (39440, 32, 32, 1), (39440, 2))
('Test set', (39440, 32, 32, 1), (39440, 2))


## Start building the network

At this point we want to start creating TensorFlow variables.

In [27]:
sess = tf.InteractiveSession()

### Variables

In [28]:
image_size = 32*32
nlabels = 2
nchannels = 1
#x  = tf.placeholder(tf.float32, shape=[None, image_size])  # 784
x = tf.placeholder(tf.float32, shape=[None, image_size, image_size, nchannels])
y_ = tf.placeholder(tf.float32, shape=[None, nlabels ])  # 10

### Function definitions

In [29]:
def weight_variable(shape):
  initial = tf.truncated_normal(shape, stddev=0.1)
  return tf.Variable(initial)

def bias_variable(shape):
  initial = tf.constant(0.1, shape=shape)
  return tf.Variable(initial)

In [30]:
def conv2d(x, W):
  return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
  return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')

In [31]:
def sample_training_data(X_in, y_in, n=100):
    
    # Generate list of integers
    ind = np.arange(y_in.shape[0])
    
    #print("Size of integer list:",len(ind))
    #print(ind)
    # Randomize this list and make new, full arrays
    np.random.shuffle(ind)
    #print(ind)
    _X = X_in[ind]
    _y = y_in[ind]
    #print
    #print("Shape of tmp _X array:",_X.shape)
    #print("Shape of tmp _Y array:",_y.shape)
    
    # Get a subset of size n of these
    X = _X[:n]
    y = _y[:n]
    return X, y

In [32]:
#xtest, ytest = sample_training_data(train_dataset, train_labels, n=50)
#print("Shape of sample X:",xtest.shape)
#print("Shape of sample y:",ytest.shape)

### First convolutional layer

In [33]:
# Compute 32 features for each of the 5x5 patches.  
# The first two dimensions are the patch size, 
# the next is the number of input channels (=1 since this a black and white image), 
# and the last is the number of output channels. 
#
# We will also have a bias vector with a component for each output channel.

W_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32])

In [34]:
# Because the input data is already set to the correct dimensions, we do not need
# to do any reshaping here
x_image = x

In [35]:
# Now run the convolution, activation function, and max pool
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)
print(h_conv1.get_shape())
print(h_pool1.get_shape())

(?, 1024, 1024, 32)
(?, 512, 512, 32)


### Second convolutional layer

In [36]:
# We again use a 5x5 patch (1st two dimensions)
# with an input layer of 32 (3rd dimension), since that was the output
#   from the previous layer
# Make this output 64 features

W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

In [37]:
print(h_conv2.get_shape())
print(h_pool2.get_shape())

(?, 512, 512, 64)
(?, 256, 256, 64)


### Third convolutional layer

In [38]:
W_conv3 = weight_variable([5, 5, 64, 128])
b_conv3 = bias_variable([128])

h_conv3 = tf.nn.relu(conv2d(h_pool2, W_conv3) + b_conv3)
h_pool3 = max_pool_2x2(h_conv3)

In [39]:
print(h_conv3.get_shape())
print(h_pool3.get_shape())

(?, 256, 256, 128)
(?, 128, 128, 128)


### Fully connected layer

In [40]:
# The first two numbers are the dimensions of the final layer (256x256)
# The 3rd number is the number of features in the final layer
# The 4th number is the number of neurons in the FLC
W_fc1 = weight_variable([128 * 128 * 128, 1024])
b_fc1 = bias_variable([1024])

In [45]:
h_pool3_flat = tf.reshape(h_pool3, [-1, 128*128*128])
h_fc1 = tf.nn.relu(tf.matmul(h_pool3_flat, W_fc1) + b_fc1)

### Dropouts

In [46]:
keep_prob  = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

### Readout Layer

In [47]:
W_fc2 = weight_variable([1024, nlabels])
b_fc2 = bias_variable([nlabels])

y_conv = tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)

### Training and Evaluating the Model

In [48]:
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_*tf.log(y_conv),reduction_indices=[1]))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

## Run the network

In [ ]:
# First initialize all variables

sess.run(tf.initialize_all_variables())

#init = tf.initialize_all_variables()
#sess = tf.Session()
#sess.run(init)

In [ ]:
batch_size = 50
for i in range(20000):
    batch_x, batch_y = sample_training_data(train_dataset, train_labels, n=batch_size)
    if i%100 == 0:
        train_accuracy = accuracy.eval(feed_dict={
            x:batch_x, y_: batch_y, keep_prob: 1.0})
        print("step %d, training accuracy %g"%(i, train_accuracy))

    
    train_step.run(feed_dict={x: batch_x, y_: batch_y, keep_prob:0.5})
    #sess.run(train_step, feed_dict={x: batch_x, y_: batch_y, keep_prob:0.5})
#print("test accuracy %g"%accuracy.eval(feed_dict={
#    x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0}))